In [23]:
from pruneshift.networks import create_network
from pruneshift.prune import prune
from pruneshift.prune_info import PruneInfo
from pytorch_lightning import seed_everything
import torch.nn as nn
import torch
import numpy as np

In [39]:
def prune_old(model, prune_ratio, prune_method):
    cfg = []
    cfg_mask = {}
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d):
            weight_copy = m.weight.data.abs().clone().cpu().numpy()
            L1_norm = np.sum(weight_copy, axis=(1,2,3))
            prune_prob_stage = prune_ratio
            num_keep = 0
            arg_max = np.argsort(L1_norm)
            new_method = np.sort(L1_norm)
            thre = new_method[int((len(new_method)-1)*prune_ratio)]           
            
            if prune_method == 'large':
                thre = new_method[int((len(new_method)-1)*(1 - prune_ratio))]
                test_mask = L1_norm <= thre
            elif prune_method == 'small':
                thre = new_method[int((len(new_method)-1)*prune_ratio)]
                test_mask = L1_norm > thre
            for i in range(weight_copy.shape[0]):
                if test_mask[i] == 0:
                    s= m.weight.data.cpu().numpy()[i].shape
                    temp = m.weight.data.cpu().numpy()[i]*np.zeros(s)
                    m.weight.data[i] = nn.Parameter(torch.from_numpy(temp).cuda())
                if torch.sum(m.weight.data[i]).item() < 0. or torch.sum(m.weight.data[i]).item() > 0. :
                    num_keep += 1
            # print('num_keep: ', num_keep)                        
            cfg_mask[name] = test_mask
            cfg.append(num_keep)
    return cfg, cfg_mask


In [3]:
dummy_input = torch.normal(mean=0., std=1., size=(1, 3, 32, 32))

In [13]:
info = prune(net, "l1_channels", 2.)

In [24]:
seed_everything(0)
net2 = create_network(group="cifar", name="resnet18", num_classes=100)
net2.fc.is_protected = True
info = prune(net2, "l1_channels", 2.)
net2(dummy_input)

tensor([[ 0.1226, -0.3242, -0.1882, -0.5156,  0.4519, -0.2777, -0.1067, -0.2936,
         -0.1102, -0.2097,  0.6799,  0.3199, -0.4316,  0.1312,  0.8116,  0.0176,
          0.3117, -0.4638,  0.3737, -0.4508, -0.0836, -0.7474,  0.4949,  0.6027,
          0.3251,  0.4565,  0.0537,  0.4929,  0.3736,  0.1732, -0.1826,  0.4629,
          0.1957,  0.2862,  0.1563, -0.1393,  0.4752,  0.1536,  0.2108, -0.2385,
          0.3725, -0.1596, -0.3806,  0.1239,  0.5592,  0.0344, -0.2755,  0.2677,
          0.1641, -0.1437, -0.2449,  0.0086, -0.1683,  0.2320,  0.3626, -0.2364,
          0.0361, -0.1872,  0.2745,  0.4183,  0.1054,  0.0286, -0.2189,  0.1334,
          0.0894,  0.2281,  0.5913, -0.3982,  0.0644,  0.4166, -0.0832, -0.0959,
         -0.3995,  0.7099, -0.2105,  0.3882,  0.0213, -0.3505, -0.4673,  0.3740,
          0.2067,  0.4556,  0.2411,  0.0492,  0.1912,  0.1908,  0.1510, -0.3922,
          0.1249, -0.4491,  0.0344,  0.3776, -0.4994, -0.2085,  0.3636,  0.1178,
          0.1024,  0.1337,  

In [54]:
mask2 = torch.tensor([channel.flatten()[0] for channel in net2.layer3[0].conv2.weight_mask])

In [53]:
# Prune method 1
seed_everything(0)
net1 = create_network(group="cifar", name="resnet18", num_classes=100)
_, cfg_mask = prune_old(net1, 0.5, "small")
net1(dummy_input)

tensor([[ 0.1433, -0.3341, -0.2094, -0.5419,  0.4520, -0.2855, -0.0950, -0.2854,
         -0.1080, -0.2225,  0.6725,  0.3309, -0.4396,  0.1074,  0.8305, -0.0065,
          0.2967, -0.4600,  0.3738, -0.4602, -0.0813, -0.7327,  0.5199,  0.5726,
          0.3677,  0.4860,  0.0581,  0.4677,  0.3677,  0.1953, -0.1763,  0.4535,
          0.2018,  0.3035,  0.1753, -0.1400,  0.5050,  0.1495,  0.2168, -0.2205,
          0.3586, -0.1519, -0.3939,  0.1257,  0.5511,  0.0130, -0.2921,  0.2850,
          0.1722, -0.1401, -0.2426,  0.0226, -0.1849,  0.2307,  0.3527, -0.2365,
          0.0400, -0.2160,  0.2800,  0.4276,  0.1006,  0.0524, -0.2373,  0.1068,
          0.0719,  0.2332,  0.6009, -0.4068,  0.0463,  0.4309, -0.0550, -0.0980,
         -0.3997,  0.6997, -0.1957,  0.4187,  0.0040, -0.3744, -0.4403,  0.3474,
          0.2051,  0.4407,  0.2542,  0.0621,  0.1865,  0.1927,  0.1255, -0.3697,
          0.1150, -0.4493,  0.0365,  0.4018, -0.5138, -0.1925,  0.3486,  0.1185,
          0.1040,  0.1435,  

In [44]:
mask1 = torch.tensor(cfg_mask["layer3.0.conv2"])

In [55]:
mask2 != mask1

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [56]:
mask2.mean()

tensor(0.4961)

In [57]:
mask1.to(dtype=torch.float).mean()

tensor(0.5000)

In [39]:
# Before prunning
net(dummy_input)

tensor([[ 0.4967, -0.6273,  0.0956, -0.3661, -0.9425, -0.0727, -0.3301,  0.2415,
         -0.0932, -0.1521, -0.4340,  0.7341,  0.0457, -0.1284, -0.8879,  0.2180,
         -0.2388,  0.0477,  0.0658,  0.2347, -0.0022, -0.2145, -0.4336, -0.1216,
          0.1928, -1.2796,  0.0931,  0.0168,  0.6098,  0.6294,  0.4161, -0.5960,
          0.6196, -0.6782, -0.0661, -0.4142,  0.2420,  0.1813,  0.0243,  0.9909,
         -0.1120,  0.0661,  0.3912, -0.1665,  0.7819,  0.0261,  0.6606,  0.2913,
         -0.5605,  0.1330, -1.1283,  0.2660,  0.3874, -0.0600, -0.2499, -0.4945,
          0.3295, -0.4229,  0.6305, -0.1965,  0.1622, -0.0285,  0.0255,  0.2599,
          0.4716,  1.1069, -0.2047,  0.5769, -0.0098,  0.3264, -0.5929,  0.8277,
         -0.1660,  0.3703, -0.3028,  0.4326, -0.4192, -0.5827, -0.0222, -0.2529,
         -0.2748, -0.3862, -0.1498, -1.0247, -0.1305,  0.6265,  0.5201,  0.4304,
          0.3221, -0.3897,  0.1329, -0.2765,  0.5442,  0.2728, -0.8835, -0.0062,
          0.1715, -0.1010, -